#### Importações

In [1]:
import pandas as pd
import matplotlib.pyplot as plt

# Configurações para exibição do DataFrame
pd.set_option('display.max_columns', None)  # Mostrar todas as colunas
pd.set_option('display.max_rows', None)     # Mostrar todas as linhas
pd.set_option('display.max_colwidth', None) # Mostrar todo o conteúdo das células

#### Importando os dataframes que serão analisados

In [2]:
df_original = pd.read_csv("../dados/nova_plataforma.csv")
df_gan = pd.read_csv("../dados/registros_gan.csv")

##### Removendo as colunas de df_original que não estão na df_gan e df_regessao

In [3]:
colunas_para_remover = ['platform', 'recommended (1 partial; 2 complete)', 'impacts_of_option-partial']
df_original.drop(colunas_para_remover, axis=1, inplace=True)

#### Analisando os registros gerados pela gan

##### Concatenado as estatisticas obtidas do df_original e df_gan

In [4]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [5]:
original_x_gan.head(25)

mean_original  \
water_depth (m)                                      124.428571   
weight (t)                                         18815.714286   
installation_date                                     38.857143   
type_of_production (1 oil and gas; 2 oil; 3 gas)       1.571429   
number_of_legs                                         7.428571   
number_of_piles                                       17.142857   
height_of_jacket_or_sub-structure (m)                137.500000   
distance_to_coast (km)                               218.571429   
risk_to_other_users-complete                           0.428571   
risk_to_other_users-partial                            0.000007   
risk_to_personnel-complete                             0.149286   
risk_to_personnel-partial                              0.082857   
energy_consumption-complete (GJ)                  685558.428571   
energy_consumption-partial (GJ)                   618058.571429   
emissions-complete (t)                             54634.714286   
emissions-partial (t)                              45170.142857   
impacts_of_option-complete                             0.237143   
technical_feasibility_or_challenge-complete            0.227143   
technical_feasibility_or_challenge-partial             0.835714   
commercial_impact_on_fisheries-complete                0.624286   
commercial_impact_on_fisheries-partial                 0.562857   
wider_community_impact-complete                        0.500000   
wider_community_impact-partial                         0.500000   
total_removal_cost-complete                            0.876000   
total_removal_cost-partial                             0.645143   

                                                       mean_gan  \
water_depth (m)                                      122.320300   
weight (t)                                         17046.900400   
installation_date                                     36.312900   
type_of_production (1 oil and gas; 2 oil; 3 gas)       1.377200   
number_of_legs                                         6.465200   
number_of_piles                                       17.522300   
height_of_jacket_or_sub-structure (m)                138.810120   
distance_to_coast (km)                               234.433200   
risk_to_other_users-complete                           0.561100   
risk_to_other_users-partial                            0.000005   
risk_to_personnel-complete                             0.141448   
risk_to_personnel-partial                              0.081217   
energy_consumption-complete (GJ)                  671142.821300   
energy_consumption-partial (GJ)                   560017.730400   
emissions-complete (t)                             49234.004900   
emissions-partial (t)                              43896.622000   
impacts_of_option-complete                             0.180232   
technical_feasibility_or_challenge-complete            0.159905   
technical_feasibility_or_challenge-partial             0.874297   
commercial_impact_on_fisheries-complete                0.667558   
commercial_impact_on_fisheries-partial                 0.630912   
wider_community_impact-complete                        0.470872   
wider_community_impact-partial                         0.520203   
total_removal_cost-complete                            0.904364   
total_removal_cost-partial                             0.505903   

                                                   std_original  \
water_depth (m)                                       22.307377   
weight (t)                                          5472.264453   
installation_date                                      5.639993   
type_of_production (1 oil and gas; 2 oil; 3 gas)       0.786796   
number_of_legs                                         1.511858   
number_of_piles                                       10.106575   
height_of_jacket_or_sub-structure (m)                 18.993420   
distance_to_coast (km)      

#### Removendo registros que contém células que ultrapassam o intervalo da coluna (min e max)

In [6]:
def ultrapassou_intervalo(registro, coluna, df_original):
    if registro > df_original[coluna].max() or registro < df_original[coluna].min():
        return True
    else:
        return False

In [7]:
def remover_registros_fora_do_intervalo(df_gan, df_original):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            if ultrapassou_intervalo(linha[coluna], coluna, df_original):
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro
    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [8]:
df_gan_limpo = remover_registros_fora_do_intervalo(df_gan, df_original)

0


In [9]:
df_gan_limpo.shape

(10000, 25)

#### Removendo outliers com base na amplitude interquartil

In [10]:
def remover_outliers_iqr(df_gan):
    indices_para_remover = []

    for indice, linha in df_gan.iterrows():
        for coluna in df_gan.columns:
            q1 = df_gan[coluna].quantile(0.25)
            q3 = df_gan[coluna].quantile(0.75)
            
            iqr = q3 - q1 # Amplitude interquantil
            limite_superior = q3 + 1.5 * iqr
            limite_inferior = q1 - 1.5 * iqr

            if linha[coluna] < limite_inferior or linha[coluna] > limite_superior:
                indices_para_remover.append(indice)
                break  # Se um valor exceder o intervalo, já podemos excluir o registro inteiro

    df_gan_limpo = df_gan.drop(indices_para_remover)
    print(len(indices_para_remover))
    return df_gan_limpo

In [11]:
#df_gan_limpo = remover_outliers_iqr(df_gan)

In [12]:
df_gan.shape, df_gan_limpo.shape

((10000, 25), (10000, 25))

In [13]:
#df_gan_limpo.duplicated().sum()

#### Comparando novamente as bases de dados

In [14]:
# Obtendo estatísticas para df_original e df_gan
describe_original = df_original.describe().transpose()
describe_gan = df_gan_limpo.describe().transpose()

# Removendo a coluna count, pois nesse caso apenas diz a quantidade de registros. 
# Portanto, não precisamos dela para a comparação
describe_original.drop(columns='count', inplace=True)
describe_gan.drop(columns='count', inplace=True)

# Adicionando sufixos aos nomes das colunas para distinguir entre os dois DataFrames
describe_original = describe_original.add_suffix('_original')
describe_gan = describe_gan.add_suffix('_gan')

# Criando DataFrame vazio para armazenar estatísticas intercaladas
original_x_gan = pd.DataFrame()

# Intercalando as colunas dos DataFrames originais e sintéticos
for coluna_original, coluna_gan in zip(describe_original.columns, describe_gan.columns):
    original_x_gan[coluna_original] = describe_original[coluna_original]
    original_x_gan[coluna_gan] = describe_gan[coluna_gan]


In [15]:
original_x_gan.head(26)

mean_original  \
water_depth (m)                                      124.428571   
weight (t)                                         18815.714286   
installation_date                                     38.857143   
type_of_production (1 oil and gas; 2 oil; 3 gas)       1.571429   
number_of_legs                                         7.428571   
number_of_piles                                       17.142857   
height_of_jacket_or_sub-structure (m)                137.500000   
distance_to_coast (km)                               218.571429   
risk_to_other_users-complete                           0.428571   
risk_to_other_users-partial                            0.000007   
risk_to_personnel-complete                             0.149286   
risk_to_personnel-partial                              0.082857   
energy_consumption-complete (GJ)                  685558.428571   
energy_consumption-partial (GJ)                   618058.571429   
emissions-complete (t)                             54634.714286   
emissions-partial (t)                              45170.142857   
impacts_of_option-complete                             0.237143   
technical_feasibility_or_challenge-complete            0.227143   
technical_feasibility_or_challenge-partial             0.835714   
commercial_impact_on_fisheries-complete                0.624286   
commercial_impact_on_fisheries-partial                 0.562857   
wider_community_impact-complete                        0.500000   
wider_community_impact-partial                         0.500000   
total_removal_cost-complete                            0.876000   
total_removal_cost-partial                             0.645143   

                                                       mean_gan  \
water_depth (m)                                      122.320300   
weight (t)                                         17046.900400   
installation_date                                     36.312900   
type_of_production (1 oil and gas; 2 oil; 3 gas)       1.377200   
number_of_legs                                         6.465200   
number_of_piles                                       17.522300   
height_of_jacket_or_sub-structure (m)                138.810120   
distance_to_coast (km)                               234.433200   
risk_to_other_users-complete                           0.561100   
risk_to_other_users-partial                            0.000005   
risk_to_personnel-complete                             0.141448   
risk_to_personnel-partial                              0.081217   
energy_consumption-complete (GJ)                  671142.821300   
energy_consumption-partial (GJ)                   560017.730400   
emissions-complete (t)                             49234.004900   
emissions-partial (t)                              43896.622000   
impacts_of_option-complete                             0.180232   
technical_feasibility_or_challenge-complete            0.159905   
technical_feasibility_or_challenge-partial             0.874297   
commercial_impact_on_fisheries-complete                0.667558   
commercial_impact_on_fisheries-partial                 0.630912   
wider_community_impact-complete                        0.470872   
wider_community_impact-partial                         0.520203   
total_removal_cost-complete                            0.904364   
total_removal_cost-partial                             0.505903   

                                                   std_original  \
water_depth (m)                                       22.307377   
weight (t)                                          5472.264453   
installation_date                                      5.639993   
type_of_production (1 oil and gas; 2 oil; 3 gas)       0.786796   
number_of_legs                                         1.511858   
number_of_piles                                       10.106575   
height_of_jacket_or_sub-structure (m)                 18.993420   
distance_to_coast (km)      

In [16]:
describe_diferenca = (df_original.describe() - df_gan.describe()).transpose()
describe_diferenca.drop(columns='count', inplace=True)
describe_diferenca

mean           std  \
water_depth (m)                                       2.108271  4.220193e+00   
weight (t)                                         1768.813886  6.507370e+02   
installation_date                                     2.544243  1.833669e-01   
type_of_production (1 oil and gas; 2 oil; 3 gas)      0.194229  5.825394e-02   
number_of_legs                                        0.963371 -4.333843e-01   
number_of_piles                                      -0.379443  9.379050e-01   
height_of_jacket_or_sub-structure (m)                -1.310120  7.715514e+00   
distance_to_coast (km)                              -15.861771  4.567531e+00   
risk_to_other_users-complete                         -0.132529  3.824492e-02   
risk_to_other_users-partial                           0.000002  9.863048e-07   
risk_to_personnel-complete                            0.007838  3.057540e-02   
risk_to_personnel-partial                             0.001640  1.949202e-02   
energy_consumption-complete (GJ)                  14415.607271  6.630902e+04   
energy_consumption-partial (GJ)                   58040.841029  3.198525e+04   
emissions-complete (t)                             5400.709386  3.279714e+03   
emissions-partial (t)                              1273.520857  1.309282e+03   
impacts_of_option-complete                            0.056911  7.683624e-02   
technical_feasibility_or_challenge-complete           0.067238  2.151533e-02   
technical_feasibility_or_challenge-partial           -0.038583  8.534478e-02   
commercial_impact_on_fisheries-complete              -0.043272  4.097534e-02   
commercial_impact_on_fisheries-partial               -0.068055  8.101826e-02   
wider_community_impact-complete                       0.029128  1.113803e-01   
wider_community_impact-partial                       -0.020203  9.919712e-02   
total_removal_cost-complete                          -0.028364  4.932713e-02   
total_removal_cost-partial                            0.139239  8.551515e-02   

                                                     min           25%  \
water_depth (m)                                    0.000  5.000000e-01   
weight (t)                                         0.000  3.891750e+03   
installation_date                                  0.000  3.500000e+00   
type_of_production (1 oil and gas; 2 oil; 3 gas)   0.000  0.000000e+00   
number_of_legs                                     0.000  4.000000e+00   
number_of_piles                                    0.000  5.000000e-01   
height_of_jacket_or_sub-structure (m)             -0.608 -7.379750e+00   
distance_to_coast (km)                             0.000 -2.000000e+00   
risk_to_other_users-complete                       0.000  0.000000e+00   
risk_to_other_users-partial                        0.000 -1.494288e-10   
risk_to_personnel-complete                         0.000  1.000000e-03   
risk_to_personnel-partial                          0.000 -1.200000e-02   
energy_consumption-complete (GJ)                 -21.000  5.842900e+04   
energy_consumption-partial (GJ)                    0.000  7.716000e+04   
emissions-complete (t)                            -1.000  1.080300e+04   
emissions-partial (t)                              0.000  3.474000e+03   
impacts_of_option-complete                         0.000  0.000000e+00   
technical_feasibility_or_challenge-complete        0.000  0.000000e+00   
technical_feasibility_or_challenge-partial         0.000 -5.075000e-02   
commercial_impact_on_fisheries-complete            0.000  1.580000e-01   
commercial_impact_on_fisheries-partial             0.000 -6.300000e-02   
wider_community_impact-complete                    0.000  2.400000e-02   
wider_community_impact-partial                     0.000 -2.000000e-02   
total_removal_cost-complete                        0.000 -8.900000e-02   
total_removal_cost-partial                         0.000  1.420000e-01   

                                                    

#### Analisando a semelhança dos dataframe com base no KSComplement e CorrelationSimilarity

In [17]:
from sdmetrics.reports.single_table import QualityReport
from sdv.metadata import SingleTableMetadata

In [18]:
df_original.head()

water_depth (m)  weight (t)  installation_date  \
0               99       22000                 36   
1              112       20364                 41   
2              141       15561                 46   
3              156       27647                 44   
4              103       18584                 33   

   type_of_production (1 oil and gas; 2 oil; 3 gas)  number_of_legs  \
0                                                 1               8   
1                                                 1               8   
2                                                 2               8   
3                                                 2               8   
4                                                 1               8   

   number_of_piles  height_of_jacket_or_sub-structure (m)  \
0                8                                  114.0   
1                9                                  123.0   
2               26                                  147.0   
3               32                                  166.0   
4               20                                  122.5   

   distance_to_coast (km)  risk_to_other_users-complete  \
0                     269                             1   
1                     259                             1   
2                     120                             0   
3                     240                             0   
4                     230                             0   

   risk_to_other_users-partial  risk_to_personnel-complete  \
0                 0.000000e+00                       0.290   
1                 0.000000e+00                       0.320   
2                 2.300000e-05                       0.025   
3                 1.500000e-05                       0.040   
4                 7.800000e-08                       0.090   

   risk_to_personnel-partial  energy_consumption-complete (GJ)  \
0                       0.12                           1110100   
1                       0.16                           1180500   
2                       0.01                            297654   
3                       0.02                            487750   
4                       0.06                            733082   

   energy_consumption-partial (GJ)  emissions-complete (t)  \
0                           817000                   87000   
1                           895500                   92000   
2                           530148                   24277   
3                           570818                   40416   
4                           511765                   59588   

   emissions-partial (t)  impacts_of_option-complete  \
0                  58500                        0.00   
1                  64000                        0.00   
2                  31064                        0.66   
3                  45266                        1.00   
4                  41170                        0.00   

   technical_feasibility_or_challenge-complete  \
0                                         0.00   
1                                         0.00   
2                                         0.25   
3                                         0.50   
4                                         0.39   

   technical_feasibility_or_challenge-partial  \
0                                        1.00   
1                                        1.00   
2                                        1.00   
3                                        1.00   
4                                        0.15   

   commercial_impact_on_fisheries-complete  \
0                                     0.79   
1                                     0.79   
2                                     1.00   
3                                     1.00   
4                                     0.00   

   commercial_impact_on_fisheries-partial  wider_community_impact-complete  \
0                                    0.78                              0.5   
1                         

In [19]:
df_gan_limpo.head()

water_depth (m)  weight (t)  installation_date  \
0              105       20869                 38   
1              148       19612                 45   
2              108       12805                 31   
3              115       15067                 31   
4              103       17639                 31   

   type_of_production (1 oil and gas; 2 oil; 3 gas)  number_of_legs  \
0                                                 1               8   
1                                                 2               8   
2                                                 1               4   
3                                                 2               8   
4                                                 1               8   

   number_of_piles  height_of_jacket_or_sub-structure (m)  \
0               15                                120.049   
1               29                                144.482   
2               11                                139.998   
3               26                                148.716   
4               22                                126.219   

   distance_to_coast (km)  risk_to_other_users-complete  \
0                     278                             1   
1                     127                             0   
2                     281                             1   
3                     269                             0   
4                     278                             0   

   risk_to_other_users-partial  risk_to_personnel-complete  \
0                 2.401143e-09                       0.295   
1                 2.145316e-05                       0.078   
2                 4.023612e-10                       0.144   
3                 4.834503e-09                       0.075   
4                 7.060170e-11                       0.166   

   risk_to_personnel-partial  energy_consumption-complete (GJ)  \
0                      0.134                           1134187   
1                      0.038                            426483   
2                      0.092                            801732   
3                      0.098                            556419   
4                      0.127                            918966   

   energy_consumption-partial (GJ)  emissions-complete (t)  \
0                           860060                   89030   
1                           581078                   42009   
2                           466598                   46870   
3                           440718                   44149   
4                           490582                   79732   

   emissions-partial (t)  impacts_of_option-complete  \
0                  62573                       0.000   
1                  53709                       0.563   
2                  36178                       0.000   
3                  33555                       0.000   
4                  45606                       0.000   

   technical_feasibility_or_challenge-complete  \
0                                        0.000   
1                                        0.466   
2                                        0.001   
3                                        0.499   
4                                        0.435   

   technical_feasibility_or_challenge-partial  \
0                                       0.995   
1                                       0.980   
2                                       0.990   
3                                       0.172   
4                                       0.207   

   commercial_impact_on_fisheries-complete  \
0                                    0.946   
1                                    0.773   
2                                    0.916   
3                                    0.000   
4                                    0.001   

   commercial_impact_on_fisheries-partial  wider_community_impact-complete  \
0                                   0.889                            0.593   
1                         

In [20]:
# Supondo que A e B são seus DataFrames reais e sintéticos
A = df_original.copy()
B = df_gan.copy()

# # Criar o metadata
# metadata = SingleTableMetadata()
# metadata.detect_from_dataframe(data=A)
# metadata_dict = metadata.to_dict()
metadata_dict = {
    "columns": {
        "water_depth (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "id"
        },
        "weight (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "installation_date": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "type_of_production (1 oil and gas; 2 oil; 3 gas)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_legs": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "number_of_piles": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "height_of_jacket_or_sub-structure (m)": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "distance_to_coast (km)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "risk_to_other_users-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_other_users-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "risk_to_personnel-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "energy_consumption-complete (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "energy_consumption-partial (GJ)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "emissions-complete (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "emissions-partial (t)": {
            "type": "numerical",
            "pii": True,
            "sdtype": "unknown"
        },
        "impacts_of_option-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "technical_feasibility_or_challenge-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "commercial_impact_on_fisheries-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "wider_community_impact-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-complete": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        },
        "total_removal_cost-partial": {
            "type": "numerical",
            "pii": False,
            "sdtype": "numerical"
        }
    },
    "primary_key": "water_depth (m)",
    "METADATA_SPEC_VERSION": "SINGLE_TABLE_V1"
}

# Gerar o relatório de qualidade
report = QualityReport()
report.generate(real_data=A, synthetic_data=B, metadata=metadata_dict)

Generating report ...

(1/2) Evaluating Column Shapes: |██████████| 25/25 [00:00<00:00, 466.06it/s]|
Column Shapes Score: 71.46%

(2/2) Evaluating Column Pair Trends: |██████████| 300/300 [00:00<00:00, 343.39it/s]|
Column Pair Trends Score: 92.59%

Overall Score (Average): 82.03%



In [21]:
report.get_score()

0.8202597001208229

In [22]:
report.get_properties()

Property     Score
0       Column Shapes  0.714603
1  Column Pair Trends  0.925916

In [23]:
report.get_details(property_name='Column Shapes')

Column        Metric     Score
0                                  installation_date  KSComplement  0.722471
1   type_of_production (1 oil and gas; 2 oil; 3 gas)  KSComplement  0.800729
2                                     number_of_legs  KSComplement  0.759157
3                                    number_of_piles  KSComplement  0.857143
4              height_of_jacket_or_sub-structure (m)  KSComplement  0.666129
5                       risk_to_other_users-complete  KSComplement  0.867471
6                        risk_to_other_users-partial  KSComplement  0.641729
7                         risk_to_personnel-complete  KSComplement  0.763686
8                          risk_to_personnel-partial  KSComplement  0.744429
9                         impacts_of_option-complete  KSComplement  0.769914
10       technical_feasibility_or_challenge-complete  KSComplement  0.743771
11        technical_feasibility_or_challenge-partial  KSComplement  0.572014
12           commercial_impact_on_fisheries-complete  KSComplement  0.698714
13            commercial_impact_on_fisheries-partial  KSComplement  0.657457
14                   wider_community_impact-complete  KSComplement  0.715886
15                    wider_community_impact-partial  KSComplement  0.717486
16                       total_removal_cost-complete  KSComplement  0.738214
17                        total_removal_cost-partial  KSComplement  0.426457

In [24]:
report.get_details(property_name='Column Pair Trends')

Column 1  \
0                                   installation_date   
1                                   installation_date   
2                                   installation_date   
3                                   installation_date   
4                                   installation_date   
5                                   installation_date   
6                                   installation_date   
7                                   installation_date   
8                                   installation_date   
9                                   installation_date   
10                                  installation_date   
11                                  installation_date   
12                                  installation_date   
13                                  installation_date   
14                                  installation_date   
15                                  installation_date   
16                                  installation_date   
17   type_of_production (1 oil and gas; 2 oil; 3 gas)   
18   type_of_production (1 oil and gas; 2 oil; 3 gas)   
19   type_of_production (1 oil and gas; 2 oil; 3 gas)   
20   type_of_production (1 oil and gas; 2 oil; 3 gas)   
21   type_of_production (1 oil and gas; 2 oil; 3 gas)   
22   type_of_production (1 oil and gas; 2 oil; 3 gas)   
23   type_of_production (1 oil and gas; 2 oil; 3 gas)   
24   type_of_production (1 oil and gas; 2 oil; 3 gas)   
25   type_of_production (1 oil and gas; 2 oil; 3 gas)   
26   type_of_production (1 oil and gas; 2 oil; 3 gas)   
27   type_of_production (1 oil and gas; 2 oil; 3 gas)   
28   type_of_production (1 oil and gas; 2 oil; 3 gas)   
29   type_of_production (1 oil and gas; 2 oil; 3 gas)   
30   type_of_production (1 oil and gas; 2 oil; 3 gas)   
31   type_of_production (1 oil and gas; 2 oil; 3 gas)   
32   type_of_production (1 oil and gas; 2 oil; 3 gas)   
33                                     number_of_legs   
34                                     number_of_legs   
35                                     number_of_legs   
36                                     number_of_legs   
37                                     number_of_legs   
38                                     number_of_legs   
39                                     number_of_legs   
40                                     number_of_legs   
41                                     number_of_legs   
42                                     number_of_legs   
43                                     number_of_legs   
44                                     number_of_legs   
45                                     number_of_legs   
46                                     number_of_legs   
47                                     number_of_legs   
48                                    number_of_piles   
49                                    number_of_piles   
50                                    number_of_piles   
51                                    number_of_piles   
52                                    number_of_piles   
53                                    number_of_piles   
54                                    number_of_piles   
55                                    number_of_piles   
56                                    number_of_piles   
57                                    number_of_piles   
58                                    number_of_piles   
59                                    number_of_piles   
60                                    number_of_piles   
61                                    number_of_piles   
62              height_of_jacket_or_sub-structure (m)   
63              height_of_jacket_or_sub-structure (m)   
64              height_of_jacket_or_sub-structure (m)   
65              height_of_jacket_or_sub-structure (m)   
66              height_of_jacket_or_sub-structure (m)   
67              height_of_jacket_or_sub-structure (m)   
68              height_of_jacket_or_sub-structure (m)   
69              height_of_jacket_or_sub-structure (m)  